# Notebook for generate Qualitative Graphics

### Libraries import

In [ ]:
import os
import sys
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

sys.path.append("../../")

In [ ]:
from utils.savers import generate_qq_plot

### Experiment selection

In [ ]:
experiment_id = "0006"
root_path = "../../results/Ganomaly_3D/"
for i in sorted(os.listdir(root_path)):
    if experiment_id in i:
        experiment_folder = os.path.join(root_path, i)
experiment_folder

### Errors loading

In [ ]:
base_path = os.path.join(experiment_folder, "outputs/errors/")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        globals()["all_{}_{}".format(t, c)] = np.r_[[]]

for t in ["encoder", "contextual", "adversarial"]:
    for m in ["train", "val", "test"]:
        if m == "train":
            if os.path.isfile(os.path.join(base_path, t, m, "normal.npy")):
                classes = ["normal"] 
            else:
                classes = ["abnormal"]
        else:
            classes = ["normal", "abnormal"]

        for c in classes:
            all_data = "all_{}_{}".format(t, c)
            errors = np.load(os.path.join(base_path, t, m, c + ".npy"))
            globals()["{}_{}_{}".format(m, t, c)] = errors
            globals()[all_data] = np.concatenate([globals()[all_data], errors])

### Errors by patients loading

In [ ]:
base_path = os.path.join(experiment_folder, "outputs/latent_vectors/input_generator")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        globals()["all_{}_{}_patients".format(t, c)] = {}
        
for t in ["encoder", "contextual", "adversarial"]:
    for m in ["train", "val", "test"]:
        if m == "train":
            if "train_encoder_normal" in globals().keys():
                classes = ["normal"] 
            else:
                classes = ["abnormal"]
        else:
            classes = ["normal", "abnormal"]
        for c in classes:
            patients_ids_positions = [
                int(i.split("_")[1].split("-")[1].split(".")[0]) for i in sorted(
                    os.listdir(os.path.join(base_path, m, c))
                )
            ]
            all_data = "all_{}_{}_patients".format(t, c)
            data = "{}_{}_{}".format(m, t, c)
            key = "{}_{}".format(data, "patients")
            globals()[key] = {}

            for p_id in np.unique(patients_ids_positions):
                globals()[key][p_id] = []
                globals()[all_data][p_id] = []

            for i, p_id in enumerate(patients_ids_positions):
                globals()[key][p_id].append(globals()[data][i])
                globals()[all_data][p_id].append(globals()[data][i])

### Data partition distribution individual

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/individual_distributions")
if not os.path.isdir(save_path):
    os.mkdir(save_path)
for t in ["encoder", "contextual", "adversarial"]:
    for g in ["train", "val", "test", "all"]:
        if g == "train":
            classes = ["check"]
        else:
            classes = ["normal", "abnormal"]
        for cl in classes:
            if cl == "check":
                if "train_{}_normal".format(t) in globals().keys():
                    c = "normal"
                else:
                    c = "abnormal"
            else:
                c = cl
            data = globals()["{}_{}_{}".format(g, t, c)]
            h = (2*stats.iqr(data)) / (data.shape[0]**(1/3))
            n_bins = int((np.max(data) - np.min(data)) / h)
            # density = stats.gaussian_kde(data)
            plt.figure(figsize=(15,13))
            # n, x, _ = plt.hist(data, bins=n_bins, label="{} {}".format(g, c), histtype="step")
            plt.hist(data, bins=n_bins, label="{} {}".format(g, c), histtype="step", linewidth=5)
            # plt.plot(x, density(x))
            plt.xlabel("Error")#, size=14)
            plt.ylabel("Counts")#, size=14)
            plt.title("Data Distribution")
            plt.legend()
            path = os.path.join(save_path, '{}_{}_{}_distribution.png'.format(g, t, c))
            plt.savefig(path, dpi=600)
            plt.close()

### Data partition distribution mixed

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/mixed_distributions")
if not os.path.isdir(save_path):
    os.mkdir(save_path)
for t in ["encoder", "contextual", "adversarial"]:
    plt.figure(figsize=(15,13))
    for g in ["val", "test"]:
        for c in ["normal", "abnormal"]:
            data = globals()["{}_{}_{}".format(g, t, c)]
            h = (2*stats.iqr(data)) / (data.shape[0]**(1/3))
            n_bins = int((np.max(data) - np.min(data)) / h)
            plt.hist(data, bins=n_bins, label="{} {}".format(g, c), histtype="step", linewidth=5)
    plt.xlabel("Errors")#, size=14)
    plt.ylabel("Counts")#, size=14)
    plt.title("Data Distributions")
    plt.legend()
    path = os.path.join(save_path, '{}_distributions.png'.format(t))
    plt.savefig(path, dpi=600)
    plt.close()

### Data partition distribution grouped by classes

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/class_grouped_distributions")
if not os.path.isdir(save_path):
    os.mkdir(save_path)
for t in ["encoder", "contextual", "adversarial"]:
    plt.figure(figsize=(15,13))
    for g in ["val", "test", "all"]:
        for c in ["normal", "abnormal"]:
            data = globals()["{}_{}_{}".format(g, t, c)]
            h = (2*stats.iqr(data)) / (data.shape[0]**(1/3))
            n_bins = int((np.max(data) - np.min(data)) / h)
            plt.hist(data, bins=n_bins, label="{} {}".format(g, c), histtype="step", linewidth=5)
        plt.xlabel("Errors")#, size=14)
        plt.ylabel("Counts")#, size=14)
        plt.title("Data Distributions")
        plt.legend()
        path = os.path.join(save_path, '{}_{}_distributions.png'.format(g, t))
        plt.savefig(path, dpi=600)
        plt.close()

### Data partition distribution grouped by partitions

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/partition_grouped_distributions")
if not os.path.isdir(save_path):
    os.mkdir(save_path)
    
for groups in [
        ("train", "val"), ("train", "test"), ("train", "all"), 
        ("val", "test"), ("val", "all"),
        ("test", "all")
    ]:
    for t in ["encoder", "contextual", "adversarial"]:
        if groups[0] == "train":
            if "train_{}_normal".format(t) in globals().keys():
                classes = ["normal"]
            else:
                classes = ["abnormal"]
        else:
            classes = ["normal", "abnormal"]
        for c in classes:
            plt.figure(figsize=(15,13))
            for g in groups:
                data = globals()["{}_{}_{}".format(g, t, c)]
                h = (2*stats.iqr(data)) / (data.shape[0]**(1/3))
                n_bins = int((np.max(data) - np.min(data)) / h)
                plt.hist(data, bins=n_bins, label="{} {}".format(g, c), histtype="step", linewidth=5)
            plt.xlabel("Errors")#, size=14)
            plt.ylabel("Counts")#, size=14)
            plt.title("Data Distributions")
            plt.legend()
            path = os.path.join(save_path, '{}_vs_{}_{}_{}_distributions.png'.format(groups[0], groups[1], t, c))
            plt.savefig(path, dpi=600)
            plt.close()

### Data partition distribution grouped by patients

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/patients_grouped_distributions")
if not os.path.isdir(save_path):
    os.mkdir(save_path)
for t in ["encoder", "contextual", "adversarial"]:
    plt.figure(figsize=(15,13))
    for g in ["train", "val", "test"]:
        if g == "train":
            if "train_encoder_normal" in globals().keys():
                classes = ["normal"] 
            else:
                classes = ["abnormal"]
        else:
            classes = ["normal", "abnormal"]
        for c in classes:
            data = globals()["{}_{}_{}".format(g, t, c)]
            h = (2*stats.iqr(data)) / (data.shape[0]**(1/3))
            n_bins = int((np.max(data) - np.min(data)) / h)
            plt.hist(data, bins=n_bins, label="{} {}".format(g, c), histtype="step", linewidth=5)
            
            patients = globals()["{}_{}_{}_patients".format(g, t, c)]
            for p_id in patients:
                data = np.r_[patients[p_id]]
                h = (2*stats.iqr(data)) / (data.shape[0]**(1/3))
                n_bins = int((np.max(data) - np.min(data)) / h)
                plt.hist(data, bins=n_bins, label="P {}".format(p_id), histtype="step", linewidth=5)
                
            plt.xlabel("Errors")#, size=14)
            plt.ylabel("Counts")#, size=14)
            plt.title("Data Distributions")
            plt.legend()
            path = os.path.join(save_path, '{}_{}_{}_distributions.png'.format(g, t, c))
            plt.savefig(path, dpi=600)
            plt.close()

### QQ Plots

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/qq_plots")
if not os.path.isdir(save_path):
    os.mkdir(save_path)
for t in ["encoder", "contextual", "adversarial"]:
    for m in ["train", "val", "test", "all"]:
        if m == "train":
            if "train_encoder_normal" in globals().keys():
                classes = ["normal"] 
            else:
                classes = ["abnormal"]
        else:
            classes = ["normal", "abnormal"]
        for c in classes:
            data = globals()["{}_{}_{}".format(m, t, c)]
            generate_qq_plot(
                data, 
                save_path, 
                "{}_{}_{}_qqplot".format(m, t, c),
                ".png",
                np.mean(data),
                np.std(data)
            )

### Box Plots

In [ ]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/box_plots")
if not os.path.isdir(save_path):
    os.mkdir(save_path)
rows = 1
columns = 3
for m in ["val", "test", "all"]:
    fig, axs = plt.subplots(rows, columns, figsize=(15, 10))
    for i, t in enumerate(["encoder", "contextual", "adversarial"]):
        data = "{}_{}_".format(m, t)
        axs[i].boxplot([globals()[data+"normal"], globals()[data+"abnormal"]], labels=['Normal', 'Abnormal'], linewidth=2)
        axs[i].set_title("{} errors".format(t))
    filename = '{}_boxplot.png'.format(m)
    fig.savefig(os.path.join(save_path, filename), dpi=600)
    plt.close(fig)